<a href="https://colab.research.google.com/github/deliacurran/condron-lab-sleap-analysis/blob/main/sleap_training_and_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configure Training Environment

In [ ]:
# override and install python 3.10
! wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py310_22.11.1-1-Linux-x86_64.sh
! chmod +x mini.sh
! bash ./mini.sh -b -f -p /usr/local
! conda install -q -y jupyter
! conda install -q -y google-colab -c conda-forge
! python -m ipykernel install --name "py310" --user

--2025-03-28 00:27:21--  https://repo.anaconda.com/miniconda/Miniconda3-py310_22.11.1-1-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72402405 (69M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  69.05M  69.1MB/s    in 1.0s    

2025-03-28 00:27:22 (69.1 MB/s) - ‘mini.sh’ saved [72402405/72402405]

PREFIX=/usr/local
Unpacking payload ...
                                                                                   
Installing base environment...



Preparing transaction: - done
Executing transaction: | done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that y

# Install SLEAP & Dependencies

In [ ]:
# confirm using python 3.10
! python3 --version

Python 3.10.8


In [ ]:
# install sleap and dependencies:
! pip install sleap[pypi]
! pip install numpy==1.23
! pip install matplotlib-inline
! pip install ipython
! apt-get install -y libcudnn8=8.6.0.163-1+cuda11.8
! export LD_LIBRARY_PATH=/usr/local/cuda-11.8/lib64:$LD_LIBRARY_PATH
! export CUDNN_INCLUDE_DIR=/usr/local/cuda/include
! export CUDNN_LIB_DIR=/usr/local/cuda/lib64

  Using cached numpy-1.22.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0
  Using cached numpy-1.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcudnn8 is already the newest version (8.6.0.163-1+cuda11.8).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
# load sleap training files from google drive
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/path")
! unzip '002_trained_model.slp.training_job.zip'
! ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Archive:  test_trained_model.slp.training_job.zip
replace train-script.sh? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train-script.sh         
  inflating: inference-script.sh     
  inflating: centroid.json           
  inflating: jobs.yaml               
  inflating: trained_model.pkg.slp   
  inflating: centered_instance.json  
002_trained_model.slp.training_job.zip	models
centered_instance.json			test
centroid.json				test_trained_model.slp.training_job.zip
inference-script.sh			trained_model.pkg.slp
jobs.yaml				train-script.sh


# Run SLEAP Training

In [ ]:
# keep the sleap program running in background
import threading
import time
def keep_colab_alive():
    while True:
        print("Keeping Colab Session Active...")
        time.sleep(600)
threading.Thread(target=keep_colab_alive, daemon=True).start()

In [ ]:
# run sleap training using loaded training configuration
! sleap-train centroid.json trained_model.pkg.slp
! sleap-train centered_instance.json trained_model.pkg.slp

In [ ]:
# run sleap training from the last checkpoint
! sleap-train '/content/drive/path.centroid/training_config.json' '/content/drive/path/trained_model.pkg.slp'

# Run SLEAP Inference

In [ ]:
# predicting instances in suggested frames from exported .slp file
!sleap-track \
    -m #folder-prefix-in-models-folder#.centered_instance \
    -m #folder-prefix-in-models-folder#.centroid \
    --only-suggested-frames \
    -o m001.predictions.slp \
    trained_model.pkg.slp

In [ ]:
# predicting and tracking instances in uploaded video (example.mp4)
!sleap-track example.mp4 \
    --frames 0-100 \
    --tracking.tracker simple \
    -m #folder-prefix-in-models-folder#.centered_instance \
    -m #folder-prefix-in-models-folder#.centroid

# Testing

In [ ]:
# check tensorflow version and GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
! nvidia-smi

In [ ]:
# check cuda and cudnn versions
! dpkg -l | grep cudnn
! nvcc --version
! conda list cudatoolkit
from tensorflow.python.framework import ops
ops.get_default_graph()

In [ ]:
# check sleap
! python -c "import sleap; sleap.system_summary()"

# Bug Fixes

In [ ]:
# old from when I had to manually install dependencies on python 11
# switched to use python 10 so I could use pip install sleap[pypi] instead
! pip install sleap
! pip install matplotlib
! pip install cattrs
! pip install scipy
! pip install h5py
! pip install opencv-python
! pip install PyQt5
! pip install PySide2
! pip install python-rapidjson
! pip install qtpy
! conda install numpy=1.23
! pip install jsmin
! pip install tensorflow==2.10
! pip install keras>=3.5.0
# !pip install tensorflow==2.18 keras==2.13.1 opencv-python-headless
# !pip install sleap[pypi]==1.3.3

In [ ]:
# old from when I was trying to find a way to override the preinstalled cuda version 12.5 so that I could use tensorflow with GPU
# colab only supports tensorflow versions 2.1x, which requires cuda version 11.8
# used the following code to fix this problem:
  # ! apt-get install -y libcudnn8=8.6.0.163-1+cuda11.8
  # ! export LD_LIBRARY_PATH=/usr/local/cuda-11.8/lib64:$LD_LIBRARY_PATH
  # ! export CUDNN_INCLUDE_DIR=/usr/local/cuda/include
  # ! export CUDNN_LIB_DIR=/usr/local/cuda/lib64
! sudo apt update
! sudo apt install -y wget gnupg
! wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
! sudo mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
! wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-11-2_11.2.2_460.32.03-1_amd64.deb
! sudo dpkg -i cuda-11-2_11.2.2_460.32.03-1_amd64.deb
!sudo apt install cuda-11-2

In [ ]:
# old from when I was trying to find a way to override the preinstalled cuda version 12.5 (see above for more detail about error)
! pip install tensorflow-gpu==2.10
! conda install cudatoolkit
! export CUDA_HOME=/usr/local/lib/python3.10/site-packages/conda/envs/your_env_name/lib/python3.10/site-packages/cudatoolkit/11.2
! export PATH=/usr/local/lib/python3.10/site-packages/conda/envs/your_env_name/lib/python3.10/site-packages/cudatoolkit/11.2/bin:$PATH
! export LD_LIBRARY_PATH=/usr/local/lib/python3.10/site-packages/conda/envs/your_env_name/lib/python3.10/site-packages/cudatoolkit/11.2/lib64:$LD_LIBRARY_PATH
! find /usr/local/lib/python3.10/site-packages/conda/envs/your_env_name/lib/python3.10/site-packages/cudatoolkit/11.2 -name "nvcc"
! nvcc --version
! conda list cudatoolkit

In [ ]:
# old from when I was trying to find a way to override the preinstalled cuda version 12.5 (see above for more detail about error)
! sudo find /usr/local/cuda-11.2/ -name 'libcudart.so.11.0'
! sudo find / -name 'libcudart.so.11.0'
! sudo find / -name 'libcuda.so.1'
! sudo ln -s /usr/local/cuda-12.5/compat/libcuda.so.1 /usr/local/lib/libcuda.so
! sudo ln -s /usr/local/cuda-12.5/compat/libcuda.so.1 /usr/local/lib64/libcuda.so